### Passo 3 - Treinamento dos Modelos

In [ ]:
# Importação das Bibliotecas
import pickle
from io import BytesIO
import os
import pandas as pd
import optuna

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from imblearn.combine import SMOTEENN

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv

# Definição das Variáveis
load_dotenv("../.env")
connection_string = os.getenv("CONNECTION_STRING")
data_container_name = "processed"
data_blob_name = "pokemon_finalizado.csv"
model_container_name = "models"

# Conexão com o Azure Blob
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
blob_client = blob_service_client.get_blob_client(data_container_name, data_blob_name)
data_bytes = blob_client.download_blob().readall()
df = pd.read_csv(pd.io.common.BytesIO(data_bytes))

# Mapeamento das Classes de Crescimento
growth_map = {
    600000: "Fast",
    800000: "Medium Fast",
    1000000: "Medium Slow",
    1059860: "Medium Slow",
    1640000: "Slow",
    6000000: "Fluctuating"
}

df["growth_class"] = df["experience_growth"].map(growth_map)
df = df.dropna(subset=["growth_class"])

# Seleção das Features Numéricas
features = ["base_total", "capture_rate", "is_legendary"]
X = df[features]
y = df["growth_class"]

# Encoding das Classes
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Divisão dos Dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Balanceamento dos Dados
smoteenn = SMOTEENN(random_state=42)
X_train_res, y_train_res = smoteenn.fit_resample(X_train, y_train)

# Normalização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)

# Função Objetivo para o Optuna
def objective(trial):

    model_name = trial.suggest_categorical("model", ["catboost", "lightgbm", "xgboost"])

    if model_name == "catboost":
        model = CatBoostClassifier(
            iterations=trial.suggest_int("iterations", 100, 600),
            depth=trial.suggest_int("depth", 3, 10),
            learning_rate=trial.suggest_float("learning_rate", 0.01, 0.3),
            loss_function="MultiClass",
            verbose=False
        )

    elif model_name == "lightgbm":
        model = LGBMClassifier(
            n_estimators=trial.suggest_int("n_estimators", 100, 600),
            learning_rate=trial.suggest_float("learning_rate", 0.01, 0.3),
            max_depth=trial.suggest_int("max_depth", 3, 20),
            class_weight="balanced"
        )

    elif model_name == "xgboost":
        model = XGBClassifier(
            n_estimators=trial.suggest_int("n_estimators", 100, 600),
            learning_rate=trial.suggest_float("learning_rate", 0.01, 0.3),
            max_depth=trial.suggest_int("max_depth", 3, 12),
            objective="multi:softmax",
            num_class=len(label_encoder.classes_),
            eval_metric="mlogloss"
        )

    model.fit(X_train_scaled, y_train_res)
    preds = model.predict(X_test_scaled)

    f1 = f1_score(y_test, preds, average="macro")
    return f1

# Execução do Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Top 1 de Cada Modelo
best_by_model = {}

for t in study.trials:
    if t.state != optuna.trial.TrialState.COMPLETE:
        continue

    model_name = t.params["model"]

    if model_name not in best_by_model or t.value > best_by_model[model_name].value:
        best_by_model[model_name] = t

best_trials = list(best_by_model.values())

# Avaliação Final dos Modelos
final_metrics = {}

for t in best_trials:
    model_name = t.params["model"]
    params = {k: v for k, v in t.params.items() if k != "model"}

    if model_name == "catboost":
        model = CatBoostClassifier(**params, loss_function="MultiClass", verbose=False)

    elif model_name == "lightgbm":
        model = LGBMClassifier(**params, class_weight="balanced")

    elif model_name == "xgboost":
        model = XGBClassifier(**params, objective="multi:softmax", num_class=len(label_encoder.classes_), eval_metric="mlogloss")

    model.fit(X_train_scaled, y_train_res)
    preds = model.predict(X_test_scaled)

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="macro", zero_division=0)
    rec = recall_score(y_test, preds, average="macro", zero_division=0)
    f1 = f1_score(y_test, preds, average="macro", zero_division=0)

    final_metrics[model_name] = {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1_macro": f1
    }

# Exibição das Métricas Finais
final_metrics

# Seleção do Melhor Modelo Geral (maior f1_macro)
best_model_name = max(final_metrics, key=lambda m: final_metrics[m]["f1_macro"])

# Reconstrução do melhor modelo com os melhores hiperparâmetros
best_trial = best_by_model[best_model_name]
best_params = {k: v for k, v in best_trial.params.items() if k != "model"}

if best_model_name == "catboost":
    best_model = CatBoostClassifier(**best_params, loss_function="MultiClass", verbose=False)

elif best_model_name == "lightgbm":
    best_model = LGBMClassifier(**best_params, class_weight="balanced")

elif best_model_name == "xgboost":
    best_model = XGBClassifier(
        **best_params,
        objective="multi:softmax",
        num_class=len(label_encoder.classes_),
        eval_metric="mlogloss"
    )

# Treinamento final com o melhor modelo
best_model.fit(X_train_scaled, y_train_res)

# Salvando modelo + scaler + label encoder no Azure Blob
artifact_dict = {
    "model": best_model,
    "scaler": scaler,
    "label_encoder": label_encoder
}

buffer = BytesIO()
pickle.dump(artifact_dict, buffer)
buffer.seek(0)

blob_client = blob_service_client.get_blob_client(
    container=model_container_name,
    blob=f"{best_model_name}_best.pkl"
)

blob_client.upload_blob(buffer, overwrite=True)


[I 2025-11-30 17:12:01,900] A new study created in memory with name: no-name-c2a73b1b-c8d7-4442-9951-5ebfca874230


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000080 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 330
[LightGBM] [Info] Number of data points in the train set: 1145, number of used features: 2
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

[I 2025-11-30 17:12:02,453] Trial 0 finished with value: 0.3463840887599259 and parameters: {'model': 'lightgbm', 'n_estimators': 493, 'learning_rate': 0.22541633139711917, 'max_depth': 16}. Best is trial 0 with value: 0.3463840887599259.
[I 2025-11-30 17:12:03,239] Trial 1 finished with value: 0.35229664827304313 and parameters: {'model': 'catboost', 'iterations': 181, 'depth': 6, 'learning_rate': 0.04907107628993575}. Best is trial 1 with value: 0.35229664827304313.
[I 2025-11-30 17:12:04,100] Trial 2 finished with value: 0.3270935513979799 and parameters: {'model': 'catboost', 'iterations': 211, 'depth': 6, 'learning_rate': 0.058104196992370225}. Best is trial 1 with value: 0.35229664827304313.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000245 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 330
[LightGBM] [Info] Number of data points in the train set: 1145, number of used features: 2
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

[I 2025-11-30 17:12:04,576] Trial 3 finished with value: 0.35860566448801745 and parameters: {'model': 'lightgbm', 'n_estimators': 586, 'learning_rate': 0.052598767112029404, 'max_depth': 3}. Best is trial 3 with value: 0.35860566448801745.


...

[I 2025-11-30 17:13:30,639] Trial 47 finished with value: 0.3463840887599259 and parameters: {'model': 'lightgbm', 'n_estimators': 435, 'learning_rate': 0.13017663103662422, 'max_depth': 18}. Best is trial 33 with value: 0.3643885780118499.
[I 2025-11-30 17:13:31,460] Trial 48 finished with value: 0.3581154684095861 and parameters: {'model': 'xgboost', 'n_estimators': 479, 'learning_rate': 0.28343083381894296, 'max_depth': 3}. Best is trial 33 with value: 0.3643885780118499.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000032 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 330
[LightGBM] [Info] Number of data points in the train set: 1145, number of used features: 2
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

[I 2025-11-30 17:13:32,430] Trial 49 finished with value: 0.34649541634835757 and parameters: {'model': 'lightgbm', 'n_estimators': 579, 'learning_rate': 0.26608383922597884, 'max_depth': 7}. Best is trial 33 with value: 0.3643885780118499.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000069 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 330
[LightGBM] [Info] Number of data points in the train set: 1145, number of used features: 2
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posi

{'etag': '"0x8DE304CF2B6C962"',
 'last_modified': datetime.datetime(2025, 11, 30, 20, 13, 38, tzinfo=datetime.timezone.utc),
 'content_md5': bytearray(b"\'\x0b\xae\xd3<C\x96\x10:\xbf@v\xc08+\x14"),
 'client_request_id': '0cf0544c-ce29-11f0-b653-e0d55e460e19',
 'request_id': '90155005-401e-0019-7035-6219ad000000',
 'version': '2025-07-05',
 'version_id': None,
 'date': datetime.datetime(2025, 11, 30, 20, 13, 37, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None,
 'structured_body': None}